# jsonpointer lib

A biblioteca **`jsonpointer`** em Python é uma implementação da especificação **JSON Pointer (RFC 6901)**. JSON Pointer é uma sintaxe para identificar valores específicos dentro de um documento JSON, semelhante a como o XPath identifica partes de um documento XML, ou como um caminho de arquivo identifica um arquivo em um sistema de arquivos.

Basicamente, um JSON Pointer é uma string que descreve o "caminho" para um valor específico dentro de uma estrutura JSON. Ele é muito útil para:

  - Acessar partes aninhadas de um JSON de forma precisa.
  - Referenciar partes de um JSON para operações como adição, remoção ou substituição em patches JSON (JSON Patch - RFC 6902).
  - Comunicar a localização exata de um dado entre sistemas.


### Como `jsonpointer` Funciona?

A biblioteca `jsonpointer` em Python permite que você faça duas operações principais:

1.  **Avaliar um JSON Pointer:** Dado um JSON Pointer e um documento JSON, ele encontra e retorna o valor no caminho especificado.
2.  **Gerar um JSON Pointer:** Dado um documento JSON e um valor, ele pode (em alguns casos, se o valor for único no caminho) gerar o JSON Pointer correspondente.


In [1]:
%pip install jsonpointer

Note: you may need to restart the kernel to use updated packages.


In [2]:
import jsonpointer
import json
from data import json_data # data.py

In [3]:
def run_jsonpointer_example(pointer_string: str):
    """
    Executes a JSON Pointer evaluation and prints the results.
    """
    print(f"\n--- JSON Pointer: '{pointer_string}' ---")
    try:
        result = jsonpointer.resolve_pointer(json_data, pointer_string)
        print(f"  Resultado: {result}")
    except jsonpointer.JsonPointerException as e:
        print(f"  Erro ao resolver ponteiro: {e}")
    except KeyError as e:
        print(f"  Erro: Chave não encontrada - {e}")
    except IndexError as e:
        print(f"  Erro: Índice fora dos limites - {e}")


## Métodos e Propriedades Principais

A API da biblioteca `jsonpointer` é bastante concisa, focando em sua funcionalidade principal.

Vamos usar o mesmo documento JSON de exemplo para as demonstrações:

### 1. `jsonpointer.resolve_pointer(document, pointer_string)`

  - **O que faz:** A função principal para encontrar um valor em um documento JSON usando um JSON Pointer.
  - **`document`**: O objeto JSON (Python `dict` ou `list`) em que a pesquisa será realizada.
  - **`pointer_string`**: A string JSON Pointer.
  - **Retorna:** O valor encontrado no caminho especificado.
  - **Levanta:** `jsonpointer.JsonPointerException` (subclasse de `KeyError` ou `IndexError`) se o ponteiro for inválido ou o caminho não for encontrado.

**Sintaxe do JSON Pointer:**

  - Um JSON Pointer é uma string que começa com `/` (a menos que seja a string vazia `""`, que se refere ao documento inteiro).
  - Cada `/` é seguido por um "token de referência" que indica a próxima chave de objeto ou índice de array.
  - **Caracteres Especiais:** Se um token de referência contiver os caracteres `~` ou `/`, eles devem ser escapados:
      - `~` é escapado como `~0`
      - `/` é escapado como `~1`
  - **`/-`**: Quando usado como o último token de referência em um array, indica o "final do array", útil para adição de elementos (embora `jsonpointer.resolve_pointer` não adicione, é um conceito importante para JSON Patch).


In [4]:
# Ponteiro para o documento raiz (string vazia)
run_jsonpointer_example('/')

# Acessando um valor de objeto
run_jsonpointer_example('/livraria/titulo')

# Acessando um elemento de array pelo índice
run_jsonpointer_example('/livraria/livros/0') # O primeiro livro (índice 0)
run_jsonpointer_example('/livraria/livros/0/titulo') # O título do primeiro livro
run_jsonpointer_example('/livraria/livros/2/autores/1') # O segundo autor do terceiro livro

# Acessando uma chave de objeto com caracteres especiais (escapando '~' e '/')
run_jsonpointer_example('/configuracoes/chave~0com~1traco')

# Tentando acessar um caminho que não existe (levantará JsonPointerException)
run_jsonpointer_example('/livraria/livros/5/titulo') # Índice 5 não existe
run_jsonpointer_example('/usuarios/0/email') # Campo 'email' não existe


--- JSON Pointer: '/' ---
  Erro ao resolver ponteiro: member '' not found in {'livraria': {'titulo': 'Catálogo de Livros Técnicos', 'autorPrincipal': 'Dr. Tech', 'livros': [{'titulo': 'Aprendendo JSONPath', 'autores': ['Alice', 'Bob'], 'ano': 2020, 'preco': 25.5, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Dominando JMESPath', 'autores': ['Charlie'], 'ano': 2021, 'preco': 30.0, 'disponivel': False, 'editora': {'nome': 'Editora Global', 'cidade': 'Rio de Janeiro'}}, {'titulo': 'Introdução ao Web Scraping', 'autores': ['David', 'Eve', 'Frank'], 'ano': 2019, 'preco': 20.0, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Python para Iniciantes', 'autores': ['Grace'], 'ano': 2022, 'preco': 35.0, 'disponivel': True, 'tags': ['Programação', 'Iniciante']}]}, 'usuarios': [{'id': 1, 'nome': 'Ana', 'isAdmin': True, 'ultimosAcessos': ['2025-06-19', '2025-06-20']}, {'id': 2, 'nome': 'Bruno', 'isA

### 2. `jsonpointer.JsonPointerException`

Esta é a classe de exceção levantada pela biblioteca quando um JSON Pointer não pode ser resolvido. Ela é uma subclasse de `KeyError` (para chaves de objeto não encontradas) e `IndexError` (para índices de array fora dos limites). Isso permite que você capture erros específicos de acesso ao JSON.

**Exemplo de tratamento de exceção:** (Já incluído na função `run_jsonpointer_example`)

### 3. `jsonpointer.JsonPointer(pointer_string)` (Classe)

Embora `resolve_pointer()` seja a função de conveniência, você também pode instanciar um objeto `JsonPointer`. Este objeto pode ser usado para realizar a resolução.

  - **`pointer_obj = jsonpointer.JsonPointer(pointer_string)`**: Cria um objeto ponteiro.
  - **`pointer_obj.resolve(document)`**: Resolve o ponteiro no documento. É equivalente a `jsonpointer.resolve_pointer(document, pointer_string)`.
  - **`str(pointer_obj)`**: Retorna a string do ponteiro.

In [5]:
# Criando um objeto JsonPointer
pointer_obj = jsonpointer.JsonPointer('/livraria/livros/1/preco')
print(f"Objeto ponteiro criado: {pointer_obj}")
print(f"String do ponteiro: {str(pointer_obj)}")

# Resolvendo o ponteiro usando o método do objeto
preco_segundo_livro = pointer_obj.resolve(json_data)
print(f"Preço do segundo livro (usando objeto JsonPointer): {preco_segundo_livro}")

# Você pode reutilizar o objeto ponteiro para diferentes documentos
other_data = {"livraria": {"livros": [{"preco": 100.0}]}}
preco_other = jsonpointer.JsonPointer('/livraria/livros/0/preco').resolve(other_data)
print(f"Preço de outro documento: {preco_other}")

Objeto ponteiro criado: /livraria/livros/1/preco
String do ponteiro: /livraria/livros/1/preco
Preço do segundo livro (usando objeto JsonPointer): 30.0
Preço de outro documento: 100.0


### 4. `jsonpointer.set_pointer(document, pointer_string, value)`

  - **O que faz:** Define um valor em um caminho específico dentro de um documento JSON. É uma operação de modificação. Se o caminho não existir, ele tentará criá-lo.
  - **`document`**: O objeto JSON (Python `dict` ou `list`) a ser modificado.
  - **`pointer_string`**: O JSON Pointer para o local onde o valor será definido.
  - **`value`**: O novo valor a ser definido.
  - **Retorna:** O documento modificado.

In [7]:
# Criar uma cópia do json_data para não alterar o original
from exceptiongroup import catch


data_to_modify = json.loads(json.dumps(json_data))

# Modificando o título da livraria
jsonpointer.set_pointer(data_to_modify, '/livraria/titulo', 'Novo Catálogo')
print(f"Novo título da livraria: {data_to_modify['livraria']['titulo']}")

# Modificando o preço do primeiro livro
jsonpointer.set_pointer(data_to_modify, '/livraria/livros/0/preco', 29.99)
print(f"Novo preço do primeiro livro: {data_to_modify['livraria']['livros'][0]['preco']}")

# Adicionando um novo campo a um objeto (se o caminho já existir até o penúltimo nível)
jsonpointer.set_pointer(data_to_modify, '/configuracoes/idioma', 'pt-BR')
print(f"Configurações com novo idioma: {data_to_modify['configuracoes']['idioma']}")

# Adicionando um novo livro ao final do array de livros
# O token '-' indica o final do array para adição
new_book = {
    "titulo": "Programação Web com Flask",
    "autores": ["João"],
    "ano": 2024,
    "preco": 45.00,
    "disponivel": True
}
jsonpointer.set_pointer(data_to_modify, '/livraria/livros/-', new_book)
print(f"Total de livros após adição: {len(data_to_modify['livraria']['livros'])}")
print(f"Último livro adicionado: {data_to_modify['livraria']['livros'][-1]['titulo']}")

# Tentando definir em um caminho que não existe (ele cria os intermediários se forem objetos/listas)
# Cuidado: se '/novo/caminho' não existir, ele criará 'novo' como um dicionário e depois 'caminho'
# Isso pode ser útil, mas também pode levar a erros se você esperar que um índice de array seja criado em vez de um objeto.
try:
    jsonpointer.set_pointer(data_to_modify, '/novo/caminho/para/valor', 'Teste')
    print(f"\nNovo caminho criado: {data_to_modify['novo']['caminho']['para']['valor']}")
except jsonpointer.JsonPointerException as e:
    print(f"  Erro: {e}")

Novo título da livraria: Novo Catálogo
Novo preço do primeiro livro: 29.99
Configurações com novo idioma: pt-BR
Total de livros após adição: 5
Último livro adicionado: Programação Web com Flask
  Erro: member 'novo' not found in {'livraria': {'titulo': 'Novo Catálogo', 'autorPrincipal': 'Dr. Tech', 'livros': [{'titulo': 'Aprendendo JSONPath', 'autores': ['Alice', 'Bob'], 'ano': 2020, 'preco': 29.99, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Dominando JMESPath', 'autores': ['Charlie'], 'ano': 2021, 'preco': 30.0, 'disponivel': False, 'editora': {'nome': 'Editora Global', 'cidade': 'Rio de Janeiro'}}, {'titulo': 'Introdução ao Web Scraping', 'autores': ['David', 'Eve', 'Frank'], 'ano': 2019, 'preco': 20.0, 'disponivel': True, 'editora': {'nome': 'Editora Inovação', 'cidade': 'São Paulo'}}, {'titulo': 'Python para Iniciantes', 'autores': ['Grace'], 'ano': 2022, 'preco': 35.0, 'disponivel': True, 'tags': ['Programação', 'Iniciante']}, 

### 5. `jsonpointer.JsonPointerException` (Propriedades)

A exceção `JsonPointerException` tem algumas propriedades úteis para depuração:

  - `e.path`: O JSON Pointer que causou o erro.
  - `e.document`: O documento JSON onde o erro ocorreu.
  - `e.message`: A mensagem de erro detalhada.

### Casos de Uso Comuns para `jsonpointer`

  - **JSON Patch (RFC 6902)**: JSON Pointer é a base do JSON Patch, que é uma especificação para descrever um conjunto de operações (add, remove, replace, move, copy, test) a serem aplicadas a um documento JSON. A biblioteca `jsonpatch` em Python usa `jsonpointer` internamente.
  - **APIs RESTful**: Usado para referenciar partes de recursos em requisições ou respostas de API.
  - **Configurações**: Acessar valores específicos em arquivos de configuração JSON.
  - **Validação de Dados**: Identificar a localização exata de erros de validação em um documento JSON.
